In [13]:
import os
import sys
import psutil
import torch
import pickle
import yaml
import random
import numpy as np
from torch import nn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr, spearmanr
from scipy.spatial.distance import euclidean

# Add the parent directory (i.e. transformer, means parent directory of 'scripts' and 'notebooks') to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

# Import the function
from scripts.utils import *
from scripts.classes import *

In [14]:
# Load data
with open("config.yaml", "r") as config_file:
    config = yaml.safe_load(config_file)
    
model_name = config['general']['model_name']
model_family = config['output']['model_family']
model_summary_folder = config['output']['model_summary'] + model_family + model_name

# Get cpu, gpu or mps device for training.
device = select_device()

# Load data
with open(f"{model_summary_folder}/{model_name}_saved_metadata.pkl", "rb") as f:
    loaded_data = pickle.load(f)

# Convert loaded data to device
loaded_data = load_data_to_device(loaded_data, device)

 # Retrieve data
loaded_config = loaded_data["Configuration_file"]
scalers = loaded_data["Scalers"] # Dictionary
train_subjects = loaded_data["Training subjects"]
val_subjects = loaded_data["Validation subjects"]
test_subjects = loaded_data["Test subjects"]
# X_train = loaded_data["X_train"]  # PyTorch tensor
# y_train = loaded_data["y_train"]  # PyTorch tensor
# X_val = loaded_data["X_val"]      # PyTorch tensor
# y_val = loaded_data["y_val"]      # PyTorch tensor
X_test = loaded_data["X_test"]      # PyTorch tensor
y_test = loaded_data["y_test"]      # PyTorch tensor
# df_train = loaded_data["df_train"]  # Pandas DataFrame
# df_val = loaded_data["df_val"]      # Pandas DataFrame
df_test = loaded_data["df_test"] # Pandas DataFrame
training_loss = loaded_data["Training_loss"]  # Numpy array
validation_loss = loaded_data["Validation_loss"]  # Numpy array
epochs = loaded_data["Epochs"]  # Numpy array


assert loaded_config['general']['model_name'] == model_name
general = loaded_config['general']
params = loaded_config['parameters']
assert loaded_config['output']['model_family'] == model_family 
results_folder = loaded_config['output']['results'] + model_family + model_name 
d_model = loaded_config['parameters']['d_model']   # Embedding dimension
input_dim = loaded_config['parameters']['input_dim']   # 3 PPG signals (red, green, IR)
output_dim = loaded_config['parameters']['output_dim']  # 1 ECG target per time step
nhead = loaded_config['parameters']['nhead']   # Attention heads
num_layers = loaded_config['parameters']['num_layers']   # Number of transformer layers
batch_size = loaded_config['parameters']['batch_size']   # Batch size
sequence_length = loaded_config['parameters']['sequence_length'] 
num_epochs = loaded_config['parameters']['num_epochs']  
use_dataloader = loaded_config['general']['use_dataloader'] # Use a dataloader or not
random_seed = loaded_config['general']['random_seed']

print(f"Evaluating model with general settings: {general}", '\n')
print(f"Evaluating model with parameters: {params}", '\n')
print(f"Saving files to folders: {results_folder}")

Using mps device
Evaluating model with general settings: {'model_name': 'm1_autoregressive_decoding_server', 'train': True, 'eval': True, 'use_dataloader': True, 'train_shuffling': True, 'filter': 1, 'validation_set_subjects': 3, 'test_set_subjects': 3, 'random_seed': 42} 

Evaluating model with parameters: {'subject': '_all', 'action': 'all', 'sequence_length': 1000, 'sequence_step_size': 50, 'subset': 0.001, 'd_model': 6, 'input_dim': 3, 'output_dim': 1, 'nhead': 3, 'num_layers': 2, 'batch_size': 4, 'num_epochs': 300, 'learning_rate': 0.001, 'ppg_scaling_factor': 100} 

Saving files to folders: ../results/m1_ablation/m1_autoregressive_decoding_server


In [16]:
# Use a random seed for reproducibility 
seed = config['general']['random_seed']
random.seed(seed)
np.random.seed(seed) 

# Initialize the Transformer model
model = TransformerTimeSeries(input_dim=input_dim, output_dim=output_dim, d_model=d_model, nhead=nhead, num_layers=num_layers).to(device) 

# Load the model
model.load_state_dict(torch.load(f"../models/{model_family}{model_name}_trained_model.pth", map_location=torch.device(device)))

# Select cpu, gpu or mps device for evaluation
device = select_device()

### Validation
# Initialize storage for aggregated predictions and actual values
ecg_predictions = []
ecg_actuals = []
ppg = []
subjects = []  # Store subject info for each batch
actions = []   # Store action info for each batch

# Loss function: Mean Squared Error for regression tasks
loss_fn = nn.MSELoss()

# Test Loss
test_loss = np.array([])
running_test_loss = 0

# Check if the dataloader should be used or not
if use_dataloader:
    # Convert tensors to Datasets
    test_dataset = PreprocessedDataset(X_test, y_test)
    # Create DataLoaders for each set
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

Using mps device


/var/folders/ft/8d1p935913j1_tk94srvt2lm0000gn/T/ipykernel_6447/3935230863.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"../models/

In [22]:
for batch_idx, (batch_X_test, batch_y_test) in enumerate(test_loader):
    print(batch_y_test.shape)
print(batch_y_test)

torch.Size([4, 1000, 1])
torch.Size([4, 1000, 1])
torch.Size([4, 1000, 1])
torch.Size([4, 1000, 1])
torch.Size([4, 1000, 1])
torch.Size([4, 1000, 1])
torch.Size([4, 1000, 1])
torch.Size([4, 1000, 1])
torch.Size([4, 1000, 1])
torch.Size([4, 1000, 1])
torch.Size([4, 1000, 1])
torch.Size([1, 1000, 1])
tensor([[[-0.3442],
         [-0.3420],
         [-0.3397],
         [-0.3375],
         [-0.3353],
         [-0.3331],
         [-0.3310],
         [-0.3289],
         [-0.3269],
         [-0.3249],
         [-0.3230],
         [-0.3212],
         [-0.3195],
         [-0.3178],
         [-0.3162],
         [-0.3147],
         [-0.3133],
         [-0.3120],
         [-0.3108],
         [-0.3096],
         [-0.3085],
         [-0.3074],
         [-0.3064],
         [-0.3055],
         [-0.3045],
         [-0.3036],
         [-0.3026],
         [-0.3017],
         [-0.3007],
         [-0.2996],
         [-0.2985],
         [-0.2972],
         [-0.2958],
         [-0.2943],
         [-0.2926],


In [23]:
# Iterate over the validation set in batches
model.eval()  # Ensure the model is in evaluation mode
with torch.no_grad():
    if use_dataloader:
        # Iterate through the batches in the test_loader to load the data in batches
        for batch_idx, (batch_X_test, batch_y_test) in enumerate(test_loader):
            # Move the batch data to the device (GPU or CPU)
            batch_X_test = batch_X_test.to(device)
            batch_y_test = batch_y_test.to(device)
            
            # Get the start and end index of the current batch in df_test
            start_idx = batch_idx * batch_size
            end_idx = start_idx + len(batch_X_test)
            
            # Retrieve the corresponding (subject, action) pair for this batch from df_test
            batch_subjects = df_test.iloc[start_idx:end_idx]['subject'].values
            batch_actions = df_test.iloc[start_idx:end_idx]['action'].values

            # Initialize decoder input with the start token (all zeros)
            tgt_input_val = torch.zeros((batch_y_test.size(0), 1, batch_y_test.size(-1)), device=device)

            # Autoregressive decoding
            predictions = []
            for _ in range(batch_y_test.size(1)):
                # Generate tgt_mask for the current step
                tgt_mask = generate_square_subsequent_mask(tgt_input_val.size(1)).to(device)

                # Generate key padding masks
                src_key_padding_mask = (batch_X_test[:, :, 0] == 0).to(device)
                tgt_key_padding_mask = (tgt_input_val.squeeze(-1) == 0).to(device)

                # Forward pass
                step_output = model(
                    batch_X_test,
                    tgt_input_val,
                    tgt_mask=tgt_mask,
                    src_key_padding_mask=src_key_padding_mask,
                    tgt_key_padding_mask=tgt_key_padding_mask,
                )[:, -1:, :]  # Take the output for the last timestep
                predictions.append(step_output)

                # Append the prediction to tgt_input_val for the next timestep
                tgt_input_val = torch.cat([tgt_input_val, step_output], dim=1)

            # Combine all timestep predictions
            batch_predictions = torch.cat(predictions, dim=1)

            # Create the target input (tgt) for the decoder using teacher forcing
            # tgt_input_val = torch.cat(
            #     [torch.zeros((batch_y_test.size(0), 1, batch_y_test.size(-1)), device=device),  # Start token
            #     batch_y_test[:, :-1, :]],  # Shifted target sequence
            #     dim=1
            # )

            # # Generate tgt_mask 
            # # Shape: [seq_len_tgt, seq_len_tgt]
            # tgt_mask = generate_square_subsequent_mask(tgt_input_val.size(1)).to(device)

            # # Generate src_key_padding_mask 
            # # Shape: [batch_size, seq_len_src]
            # src_key_padding_mask = (batch_X_test[:, :, 0] == 0).to(device) # Use only one feature (i.e. one ppg signal)

            # # Generate tgt_key_padding_mask (pad tokens are 0 in the target)
            # # Shape: [batch_size, seq_len_tgt]
            # tgt_key_padding_mask = (tgt_input_val.squeeze(-1) == 0).to(device)

            # # Debugging: print mask shapes to ensure they are correct
            # # assert src_key_padding_mask.shape == [batch_size, sequence_length]
            # # assert src_key_padding_mask.shape == [batch_size, sequence_length]
            # # print("seq_len_src", sequence_length)
            # # print("tgt_len", sequence_length)
            # # print(f"src_key_padding_mask shape: {src_key_padding_mask.shape}")  # Should be [batch_size, seq_len_src]
            # # print(f"tgt_key_padding_mask shape: {tgt_key_padding_mask.shape}")  # Should be [batch_size, seq_len_tgt]

            # memory_mask = None

            # # Forward pass to get predictions
            # batch_predictions = model(
            #     batch_X_test, 
            #     tgt_input_val, 
            #     tgt_mask=tgt_mask, 
            #     src_key_padding_mask=src_key_padding_mask, 
            #     tgt_key_padding_mask=tgt_key_padding_mask
            # )
            # Calculate loss for this batch
            loss = loss_fn(batch_predictions, batch_y_test)

            # Accumulate total validation loss
            running_test_loss += loss.item() * batch_X_test.size(0)

            # Store predictions, actuals, subjects, and actions
            ecg_predictions.append(batch_predictions.cpu())  # Move to CPU for numpy/scaler operations
            ecg_actuals.append(batch_y_test.cpu())
            ppg.append(batch_X_test.cpu())
            subjects.extend(batch_subjects)
            actions.extend(batch_actions)

    else:
        for j in range(0, len(X_test), batch_size):
            # Get the current validation batch
            batch_X_test = X_test[j:j + batch_size].to(device)
            batch_y_test = y_test[j:j + batch_size].to(device)

            # Retrieve subject and action for the batch
            batch_subjects = df_test.iloc[j:j + batch_size]['subject'].values
            batch_actions = df_test.iloc[j:j + batch_size]['action'].values

            # Create the target input (tgt) for the decoder using teacher forcing
            tgt_input_val = torch.cat(
                [torch.zeros((batch_y_test.size(0), 1, batch_y_test.size(-1)), device=device),  # Start token
                batch_y_test[:, :-1, :]],  # Shifted target sequence
                dim=1
            )

            # Generate tgt_mask 
            # Shape: [seq_len_tgt, seq_len_tgt]
            tgt_mask = generate_square_subsequent_mask(tgt_input_val.size(1)).to(device)

            # Generate src_key_padding_mask 
            # Shape: [batch_size, seq_len_src]
            src_key_padding_mask = (batch_X_test[:, :, 0] == 0).to(device) # Use only one feature (i.e. one ppg signal)

            # Generate tgt_key_padding_mask (pad tokens are 0 in the target)
            # Shape: [batch_size, seq_len_tgt]
            tgt_key_padding_mask = (tgt_input_val.squeeze(-1) == 0).to(device)

            # Debugging: print mask shapes to ensure they are correct
            # assert src_key_padding_mask.shape == [batch_size, sequence_length]
            # assert src_key_padding_mask.shape == [batch_size, sequence_length]
            # print("seq_len_src", sequence_length)
            # print("tgt_len", sequence_length)
            # print(f"src_key_padding_mask shape: {src_key_padding_mask.shape}")  # Should be [batch_size, seq_len_src]
            # print(f"tgt_key_padding_mask shape: {tgt_key_padding_mask.shape}")  # Should be [batch_size, seq_len_tgt]

            memory_mask = None

            # Forward pass to get predictions
            batch_predictions = model(
                batch_X_test, 
                tgt_input_val, 
                tgt_mask=tgt_mask, 
                src_key_padding_mask=src_key_padding_mask, 
                tgt_key_padding_mask=tgt_key_padding_mask
            )
            # Calculate loss for this batch
            loss = loss_fn(batch_predictions, batch_y_test)

            # Accumulate total validation loss
            running_test_loss += loss.item() * batch_X_test.size(0)

            # Store predictions, actuals, subjects, and actions
            ecg_predictions.append(batch_predictions.cpu())  # Move to CPU for numpy/scaler operations
            ecg_actuals.append(batch_y_test.cpu())
            ppg.append(batch_X_test.cpu())
            subjects.extend(batch_subjects)
            actions.extend(batch_actions)

# Average the test loss over all samples
avg_test_loss = running_test_loss / len(X_test)
test_rmse = torch.sqrt(torch.tensor(avg_test_loss))
test_loss = np.append(test_loss, test_rmse.cpu())

# Concatenate all batches
ecg_predictions = torch.cat(ecg_predictions, dim=0)
ecg_actuals = torch.cat(ecg_actuals, dim=0)
ppg = torch.cat(ppg, dim=0)

# Initialize lists for original scale data
ecg_predictions_original_scale = []
ecg_actuals_original_scale = []
ppg_original_scale = []

# Process each sequence
for i in range(len(ecg_predictions)):
    # Get subject and action for the current sequence
    subject = subjects[i]
    action = actions[i]

    # Retrieve the correct scalers
    scaler_input = scalers[(subject, action)]['input_scaler']
    scaler_target = scalers[(subject, action)]['target_scaler']

    # Inverse transform predictions and actuals for the current sequence
    ecg_pred = ecg_predictions[i].squeeze(-1).numpy()  # Shape: [sequence_length]
    ecg_act = ecg_actuals[i].squeeze(-1).numpy()       # Shape: [sequence_length]
    ppg_seq = ppg[i].numpy()                          # Shape: [sequence_length, 3]

    ecg_predictions_original_scale.append(scaler_target.inverse_transform(ecg_pred.reshape(-1, 1)).flatten())
    ecg_actuals_original_scale.append(scaler_target.inverse_transform(ecg_act.reshape(-1, 1)).flatten())
    ppg_original_scale.append(scaler_input.inverse_transform(ppg_seq))

# Convert back to arrays
ecg_predictions_original_scale = np.array(ecg_predictions_original_scale)
ecg_actuals_original_scale = np.array(ecg_actuals_original_scale)
ppg_original_scale = np.array(ppg_original_scale)

# Separate PPG channels 
red_ppg = ppg_original_scale[:, :, 0]  # Red PPG
ir_ppg = ppg_original_scale[:, :, 1]   # IR PPG
green_ppg = ppg_original_scale[:, :, 2]  # Green PPG

/Users/andrifurrer/anaconda3/envs/transformer/lib/python3.12/site-packages/torch/nn/functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
### Validation
# Initialize storage for aggregated predictions and actual values
ecg_predictions = []
ecg_actuals = []
ppg = []
subjects = []  # Store subject info for each batch
actions = []   # Store action info for each batch

# Loss function: Mean Squared Error for regression tasks
loss_fn = nn.MSELoss()

# Test Loss
test_loss = np.array([])
running_test_loss = 0

# Iterate over the validation set in batches
model.eval()  # Ensure the model is in evaluation mode
with torch.no_grad():
    for j in range(0, len(X_test), batch_size):
        # Get the current validation batch
        batch_X_test = X_test[j:j + batch_size].to(device)
        batch_y_test = y_test[j:j + batch_size].to(device)

        # Retrieve subject and action for the batch
        batch_subjects = df_test.iloc[j:j + batch_size]['subject'].values
        batch_actions = df_test.iloc[j:j + batch_size]['action'].values

        # Create the target input (tgt) for the decoder using teacher forcing
        tgt_input_val = torch.cat(
            [torch.zeros((batch_y_test.size(0), 1, batch_y_test.size(-1)), device=device),  # Start token
            batch_y_test[:, :-1, :]],  # Shifted target sequence
            dim=1
        )

        # Generate tgt_mask 
        # Shape: [seq_len_tgt, seq_len_tgt]
        tgt_mask = generate_square_subsequent_mask(tgt_input_val.size(1)).to(device)

        # Generate src_key_padding_mask 
        # Shape: [batch_size, seq_len_src]
        src_key_padding_mask = (batch_X_test[:, :, 0] == 0).to(device) # Use only one feature (i.e. one ppg signal)

        # Generate tgt_key_padding_mask (pad tokens are 0 in the target)
        # Shape: [batch_size, seq_len_tgt]
        tgt_key_padding_mask = (tgt_input_val.squeeze(-1) == 0).to(device)

        # Debugging: print mask shapes to ensure they are correct
        # assert src_key_padding_mask.shape == [batch_size, sequence_length]
        # assert src_key_padding_mask.shape == [batch_size, sequence_length]
        # print("seq_len_src", sequence_length)
        # print("tgt_len", sequence_length)
        # print(f"src_key_padding_mask shape: {src_key_padding_mask.shape}")  # Should be [batch_size, seq_len_src]
        # print(f"tgt_key_padding_mask shape: {tgt_key_padding_mask.shape}")  # Should be [batch_size, seq_len_tgt]

        memory_mask = None

        # Forward pass to get predictions
        batch_predictions = model(
            batch_X_test, 
            tgt_input_val, 
            tgt_mask=tgt_mask, 
            src_key_padding_mask=src_key_padding_mask, 
            tgt_key_padding_mask=tgt_key_padding_mask
        )
        # Calculate loss for this batch
        loss = loss_fn(batch_predictions, batch_y_test)

        # Accumulate total validation loss
        running_test_loss += loss.item() * batch_X_test.size(0)

        # Store predictions, actuals, subjects, and actions
        ecg_predictions.append(batch_predictions.cpu())  # Move to CPU for numpy/scaler operations
        ecg_actuals.append(batch_y_test.cpu())
        ppg.append(batch_X_test.cpu())
        subjects.extend(batch_subjects)
        actions.extend(batch_actions)

# Average the test loss over all samples
avg_test_loss = running_test_loss / len(X_test)
test_rmse = torch.sqrt(torch.tensor(avg_test_loss))
test_loss = np.append(test_loss, test_rmse.cpu())

# Concatenate all batches
ecg_predictions = torch.cat(ecg_predictions, dim=0)
ecg_actuals = torch.cat(ecg_actuals, dim=0)
ppg = torch.cat(ppg, dim=0)

# Initialize lists for original scale data
ecg_predictions_original_scale = []
ecg_actuals_original_scale = []
ppg_original_scale = []

# Process each sequence
for i in range(len(ecg_predictions)):
    # Get subject and action for the current sequence
    subject = subjects[i]
    action = actions[i]

    # Retrieve the correct scalers
    scaler_input = scalers[(subject, action)]['input_scaler']
    scaler_target = scalers[(subject, action)]['target_scaler']

    # Inverse transform predictions and actuals for the current sequence
    ecg_pred = ecg_predictions[i].squeeze(-1).numpy()  # Shape: [sequence_length]
    ecg_act = ecg_actuals[i].squeeze(-1).numpy()       # Shape: [sequence_length]
    ppg_seq = ppg[i].numpy()                          # Shape: [sequence_length, 3]

    ecg_predictions_original_scale.append(scaler_target.inverse_transform(ecg_pred.reshape(-1, 1)).flatten())
    ecg_actuals_original_scale.append(scaler_target.inverse_transform(ecg_act.reshape(-1, 1)).flatten())
    ppg_original_scale.append(scaler_input.inverse_transform(ppg_seq))

# Convert back to arrays
ecg_predictions_original_scale = np.array(ecg_predictions_original_scale)
ecg_actuals_original_scale = np.array(ecg_actuals_original_scale)
ppg_original_scale = np.array(ppg_original_scale)

# Separate PPG channels 
red_ppg = ppg_original_scale[:, :, 0]  # Red PPG
ir_ppg = ppg_original_scale[:, :, 1]   # IR PPG
green_ppg = ppg_original_scale[:, :, 2]  # Green PPG

In [ ]:
### Normalized Evaluation metrics
# Predictions and actual values (normalized and flattened)
ecg_predictions_arr = np.array(ecg_predictions).flatten()
ecg_actuals_arr = np.array(ecg_actuals).flatten()

# Calculate the range of the actual data for normalization
actual_range_normalized = np.ptp(ecg_actuals)  # Peak-to-peak (max - min)

# Euclidean Distance
euclidean_distance_normalized = euclidean(ecg_predictions_arr, ecg_actuals_arr)

# Dynamic Time Warping (DTW)
downsampling_factor_dtw = 10
batch_size_dtw = 10
dtw_distance_normalized = compute_batched_dtw(ecg_predictions_arr, ecg_actuals_arr, batch_size_dtw, downsampling_factor_dtw)
# dtw_distance = alignment.distance

# Pearson Correlation
pearson_corr_normalized, _ = pearsonr(ecg_predictions_arr, ecg_actuals_arr)

# Spearman Correlation
spearman_corr_normalized, _ = spearmanr(ecg_predictions_arr, ecg_actuals_arr)

# Mean Squared Error (MSE)
mse_normalized = np.mean((ecg_predictions_arr - ecg_actuals_arr) ** 2)

# Mean Absolute Error (MAE)
mae_normalized = np.mean(np.abs(ecg_predictions_arr - ecg_actuals_arr))

# Root Mean Squared Error (RMSE)
rmse_normalized = np.sqrt(mse_normalized)

# Normalized Root Mean Squared Error (NRMSE)
nrmse_normalized = rmse_normalized / actual_range_normalized

# Normalized Mean Absolute Error (NMAE)
nmae_normalized = mae_normalized / actual_range_normalized

# Print metrics
metrics_normalized = {
    "Euclidean Distance": euclidean_distance_normalized,
    "DTW Distance": dtw_distance_normalized,
    "Pearson Correlation": pearson_corr_normalized,
    "Spearman Correlation": spearman_corr_normalized,
    "MSE": mse_normalized,
    "MAE": mae_normalized,
    "RMSE": rmse_normalized,
    "NRMSE": nrmse_normalized,
    "NMAE": nmae_normalized,
}

for metric, value in metrics_normalized.items():
    print(f"{metric}: {value:.4f}")

# Write metrics to a file
metrics_file = os.path.join(results_folder, f"metrics_s{config['parameters']['subject']}_{config['parameters']['action']}.txt")
with open(metrics_file, "w") as f:
    for metric, value in metrics_normalized.items():
        f.write(f"{metric}: {value:.4f}\n")

print(f"Metrics written to {metrics_file}")

In [ ]:
### Evaluation metrics
# Predictions and actual values (already scaled back to original scale)
ecg_predictions_original_scale_flattened = ecg_predictions_original_scale.flatten()  # Flatten to 1D if necessary
ecg_actuals_original_scale_flattened = ecg_actuals_original_scale.flatten()

# Calculate the range of the actual data for normalization
actual_range = np.ptp(ecg_actuals_original_scale)  # Peak-to-peak (max - min)

# Euclidean Distance
euclidean_distance = euclidean(ecg_predictions_original_scale_flattened, ecg_actuals_original_scale_flattened)

# Dynamic Time Warping (DTW)
downsampling_factor_dtw = 10
batch_size_dtw = 10 
dtw_distance = compute_batched_dtw(ecg_predictions_original_scale_flattened, ecg_actuals_original_scale_flattened, batch_size_dtw, downsampling_factor_dtw)
# dtw_distance = alignment.distance

# Pearson Correlation
pearson_corr, _ = pearsonr(ecg_predictions_original_scale_flattened, ecg_actuals_original_scale_flattened)

# Spearman Correlation
spearman_corr, _ = spearmanr(ecg_predictions_original_scale_flattened, ecg_actuals_original_scale_flattened)

# Mean Squared Error (MSE)
mse = np.mean((ecg_predictions_original_scale_flattened - ecg_actuals_original_scale_flattened) ** 2)

# Mean Absolute Error (MAE)
mae = np.mean(np.abs(ecg_predictions_original_scale_flattened - ecg_actuals_original_scale_flattened))

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Normalized Root Mean Squared Error (NRMSE)
nrmse = rmse / actual_range

# Normalized Mean Absolute Error (NMAE)
nmae = mae / actual_range

# Print metrics
metrics = {
    "Euclidean Distance": euclidean_distance,
    "DTW Distance": dtw_distance,
    "Pearson Correlation": pearson_corr,
    "Spearman Correlation": spearman_corr,
    "MSE": mse,
    "MAE": mae,
    "RMSE": rmse,
    "NRMSE": nrmse,
    "NMAE": nmae,
}

for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Write metrics to a file
metrics_file = os.path.join(results_folder, f"metrics_s{config['parameters']['subject']}_{config['parameters']['action']}.txt")
with open(metrics_file, "w") as f:
    for metric, value in metrics.items():
        f.write(f"{metric}: {value:.4f}\n")

print(f"Metrics written to {metrics_file}")

In [ ]:
### Creating graphs
# Randomly select an index from the validation data
plt.figure(figsize=(10, 5))
plt.plot(epochs, training_loss,  label='Training Loss')
plt.plot(epochs, validation_loss, label='Validation Loss')
plt.yscale('log')
plt.title(f"Training and Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
#plt.xticks(epochs)
plt.legend()

plt.savefig(f"{results_folder}/losses_s{config['parameters']['subject']}_{config['parameters']['action']}.png")

In [ ]:
    # Plot the losses
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, training_loss, label="Training Loss", marker='o')
    plt.plot(epochs, validation_loss, label="Validation Loss", marker='o')
    plt.plot(epochs, test_losses, label="Test Loss", linestyle='--', color='red')

    # Add labels, title, legend
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training, Validation, and Test Loss")
    plt.legend()

    plt.savefig(f"{results_folder}/test_loss_s{config['parameters']['subject']}_{config['parameters']['action']}.png")

In [ ]:
random_index = np.random.randint(0, len(ecg_predictions_original_scale))
ppg_scaling_factor = config['parameters']['ppg_scaling_factor']

# Select the corresponding actual and predicted ECG signals
ecg_predictions_random = ecg_predictions_original_scale[random_index]  # Predicted ECG signal
ecg_actuals_random = ecg_actuals_original_scale[random_index]  # Actual ECG signal

# Set the opacity value of alpha for the ppg signals
alpha = 0.3

# Plot the actual and predicted ECG
plt.figure(figsize=(10, 5))
plt.plot(ecg_actuals_random, label='Actual ECG')
plt.plot(ecg_predictions_random, label='Predicted ECG')
plt.title(f"ECG Prediction vs Actual (Sequence {random_index})")
plt.xlabel('Time Step')
plt.ylabel('ECG Signal')
plt.legend()

plt.savefig(f"{results_folder}/random_seq_s{config['parameters']['subject']}_{config['parameters']['action']}.png")

In [ ]:
# Plot the actual and predicted ECG with the input ppg signals
plt.figure(figsize=(10, 5))
plt.plot(ecg_actuals_random, label='Actual ECG')
plt.plot(ecg_predictions_random, label='Predicted ECG')
plt.plot(ppg_scaling_factor*red_ppg[random_index], label="Red PPG", alpha=alpha)
plt.plot(ppg_scaling_factor*ir_ppg[random_index], label="IR PPG", alpha=alpha)
plt.plot(ppg_scaling_factor*green_ppg[random_index], label="Green PPG", alpha=alpha)
plt.title(f"ECG Prediction vs Actual (Sequence {random_index}) with PPG signals")
plt.xlabel('Time Step')
plt.ylabel('ECG Signal')
plt.legend()

plt.savefig(f"{results_folder}/random_seq_s{config['parameters']['subject']}_{config['parameters']['action']}_ppg.png")

In [ ]:
# Number of random sequences to plot
num_sequences = 1

# Create a some more random sequences
plt.figure(figsize=(10, 6))

for _ in range(num_sequences):
    random_index = np.random.randint(0, len(ecg_predictions_original_scale))
    
    # Select the corresponding actual and predicted ECG signals
    ecg_predictions_random = ecg_predictions_original_scale[random_index]  # Predicted ECG signal
    ecg_actuals_random = ecg_actuals_original_scale[random_index]  # Actual ECG signal

    # Plot both actual and predicted ECG
    plt.plot(ecg_actuals_random, label=f'Actual ECG {random_index}')
    plt.plot(ecg_predictions_random, label=f'Predicted ECG {random_index}', linestyle='dashed')

plt.title("ECG Predictions vs Actual for Random Sequences")
plt.xlabel('Time Step')
plt.ylabel('ECG Signal')
plt.legend()

plt.savefig(f"{results_folder}/mult_seq_s{config['parameters']['subject']}_{config['parameters']['action']}.png")

# Create a some more random sequences with the input ppg signals
plt.figure(figsize=(10, 6))

for _ in range(num_sequences):
    random_index = np.random.randint(0, len(ecg_predictions_original_scale))
    
    # Select the corresponding actual and predicted ECG signals
    ecg_predictions_random = ecg_predictions_original_scale[random_index]  # Predicted ECG signal
    ecg_actuals_random = ecg_actuals_original_scale[random_index]  # Actual ECG signal

    # Plot both actual and predicted ECG
    plt.plot(ecg_actuals_random, label=f'Actual ECG {random_index}')
    plt.plot(ecg_predictions_random, label=f'Predicted ECG {random_index}', linestyle='dashed')
    plt.plot(ppg_scaling_factor*red_ppg[random_index], label="Red PPG", alpha=0.25)
    plt.plot(ppg_scaling_factor*ir_ppg[random_index], label="IR PPG", alpha=0.25)
    plt.plot(ppg_scaling_factor*green_ppg[random_index], label="Green PPG", alpha=0.25)

plt.title("ECG Predictions vs Actual for Random Sequences with PPG signals")
plt.xlabel('Time Step')
plt.ylabel('ECG Signal')
plt.legend()

plt.savefig(f"{results_folder}/mult_seq_s{config['parameters']['subject']}_{config['parameters']['action']}_ppg.png")

# Calculate the average ECG for both actual and predicted
ecg_predictions_average = np.mean(ecg_actuals_original_scale, axis=0)  # Average over all predictions
ecg_actuals_average = np.mean(ecg_predictions_original_scale, axis=0)  # Average over all sequences

# Plot the average ECG
plt.figure(figsize=(10, 5))
plt.plot(ecg_actuals_average, label='Average Actual ECG')
plt.plot(ecg_predictions_average, label='Average Predicted ECG')
plt.title("Average ECG Prediction vs Actual")
plt.xlabel('Time Step')
plt.ylabel('ECG Signal')
plt.legend()

plt.savefig(f"{results_folder}/average_all_seq_s{config['parameters']['subject']}_{config['parameters']['action']}.png")